# Parent-Child Chunk 를 OpenSearch 에 저장

이 노트북은 전체 문서를 Parent, Child 문서로 구분을 하여 오픈 서치 벡터스토어에 저장을 하고, 테스트를 하는 노트북 입니다. 또한 일부 문서로 검증 인덱스를 생성하여 테스트 용으로 만들기 위한 노트북 입니다.

---

## [중요] 사전 실행 노트북
이 노트북은 아래 두개의 셋업 노트북이 먼저 실행이 되어야 합니다.
- (1) Setup 노트북
    - 경로는 aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/00_setup/setup.ipynb 와 같습니다.
    -  [Setup Notebook](https://github.com/aws-samples/aws-ai-ml-workshop-kr/blob/master/genai/aws-gen-ai-kr/00_setup/setup.ipynb)
- (2) Amazon OpenSearch 설치 노트북    
    - 경로는 aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/00_setup/setup_opensearch.ipynb 와 같습니다.
    - [Setup OpenSearch](https://github.com/aws-samples/aws-ai-ml-workshop-kr/blob/master/genai/aws-gen-ai-kr/00_setup/setup_opensearch.ipynb)

# 1. Bedrock Client 생성

In [1]:
%load_ext autoreload
%autoreload 2

import sys, os
def add_python_path(module_path):
    if os.path.abspath(module_path) not in sys.path:
        sys.path.append(os.path.abspath(module_path))
        print(f"python path: {os.path.abspath(module_path)} is added")
    else:
        print(f"python path: {os.path.abspath(module_path)} already exists")
    print("sys.path: ", sys.path)

module_path = ".."
add_python_path(module_path)


python path: /home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/02_qa_chatbot is added
sys.path:  ['/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/02_qa_chatbot/01_preprocess_docs', '/opt/conda/lib/python310.zip', '/opt/conda/lib/python3.10', '/opt/conda/lib/python3.10/lib-dynload', '', '/opt/conda/lib/python3.10/site-packages', '/home/sagemaker-user/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/02_qa_chatbot']


In [3]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from local_utils import bedrock, print_ww
from local_utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print(colored("\n== FM lists ==", "green"))
pprint(bedrock_info.get_list_fm_models())

Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Light': 'amazon.titan-text-lite-v1'}


# 2. Embedding 모델 로딩

## Embedding Model 선택

In [4]:
Use_Titan_Embedding = True
Use_Cohere_English_Embedding = False

## Embedding Model 로딩

In [5]:
# We will be using the Titan Embeddings Model to generate our Embeddings.
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

if Use_Titan_Embedding:
    llm_emb = BedrockEmbeddings(client=boto3_bedrock, model_id = "amazon.titan-embed-text-v1")
    dimension = 1536
elif Use_Cohere_English_Embedding:
    llm_emb = BedrockEmbeddings(client=boto3_bedrock, model_id = "cohere.embed-english-v3")    
    dimension = 1024
else:
    lim_emb = None

llm_emb

BedrockEmbeddings(client=<botocore.client.BedrockRuntime object at 0x7fbb7327a590>, region_name=None, credentials_profile_name=None, model_id='amazon.titan-embed-text-v1', model_kwargs=None, endpoint_url=None, normalize=False)

# 3. Load all Json files

In [6]:
from local_utils.proc_docs import get_load_json, show_doc_json

In [7]:
import glob

# Specify the directory and file pattern for .txt files
folder_path = 'data/poc/preprocessed_json/all_processed_data.json'

# List all .txt files in the specified folder
json_files = glob.glob(folder_path)
# json_files = ['data/poc/customer_EFOTA.json']

# Load each item per json file and append to a list
doc_json_list = []
for file_path in json_files:
    doc_json = get_load_json(file_path)
    doc_json_list.append(doc_json)

print("all json files: ", len(doc_json_list))    
# Flatten the list of lists into a single list
all_docs = []
for item in doc_json_list:
        all_docs.extend(item)
        
print("all items: ", len(all_docs))

.[]
all json files:  1
all items:  1732


# 4. Index 생성

## Index 이름 결정

In [8]:
index_name = "v24-genai-poc-knox-parent-doc-retriever"

## Index 스키마 정의

In [9]:
index_body = {
    'settings': {
        'index': {
            'knn': True,
            'knn.space_type': 'cosinesimil'  # Example space type
        }
    },
    'mappings': {
        'properties': {
            'metadata': {
                'properties': {
                               'source' : {'type': 'keyword'},
                               'family_tree' : {'type': 'keyword'},                                        
                               'parent_id' : {'type': 'keyword'},                    
                               'last_updated': {'type': 'date'},
                               'project': {'type': 'keyword'},
                               'seq_num': {'type': 'long'},
                               'title': {'type': 'text'},  # For full-text search
                               'url': {'type': 'text'},  # For full-text search
                            }
            },            
            'text': {
                'type': 'text'
            },
            'vector_field': {
                'type': 'knn_vector',
                'dimension': f"{dimension}"  # Replace with your vector dimension
            }
        }
    }
}


# 5. LangChain OpenSearch VectorStore 생성 
## 선수 조건


## 오픈 서치 도메인 및 인증 정보 세팅

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.

In [10]:
from local_utils.proc_docs import get_parameter

In [11]:
import boto3
ssm = boto3.client('ssm', 'us-east-1')

opensearch_domain_endpoint = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_domain_endpoint',
)

opensearch_user_id = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_userid',
)

opensearch_user_password = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_password',
)


In [12]:
opensearch_domain_endpoint = opensearch_domain_endpoint
rag_user_name = opensearch_user_id
rag_user_password = opensearch_user_password

http_auth = (rag_user_name, rag_user_password) # Master username, Master password

## OpenSearch Client 생성

In [14]:
from local_utils.opensearch import opensearch_utils

In [15]:
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

## 오픈 서치 인덱스 생성 
- 오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [16]:
from local_utils.opensearch import opensearch_utils

In [17]:

index_exists = opensearch_utils.check_if_index_exists(
    os_client,
    index_name
)

if index_exists:
    opensearch_utils.delete_index(
        os_client,
        index_name
    )

opensearch_utils.create_index(os_client, index_name, index_body)
index_info = os_client.indices.get(index=index_name)
print("Index is created")
pprint(index_info)

index_name=v24-genai-poc-knox-parent-doc-retriever, exists=False

Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'v24-genai-poc-knox-parent-doc-retriever'}
Index is created
{'v24-genai-poc-knox-parent-doc-retriever': {'aliases': {},
                                             'mappings': {'properties': {'metadata': {'properties': {'family_tree': {'type': 'keyword'},
                                                                                                     'last_updated': {'type': 'date'},
                                                                                                     'parent_id': {'type': 'keyword'},
                                                                                                     'project': {'type': 'keyword'},
                                                                                                     'seq_num': {'type': 'long'},
                                                                   

## 랭체인 인덱스 연결 오브젝트 생성

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

In [18]:
from langchain.vectorstores import OpenSearchVectorSearch

In [19]:
vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2",
    bulk_size=100000,
    timeout=60    
)
vector_db

# 6. Chunking JSON Doc 

## Chunk Size and Chunk Overlap Size 결정

In [20]:
parent_chunk_size = 4096
parent_chunk_overlap = 0

child_chunk_size = 1024
child_chunk_overlap = 256

opensearch_parent_key_name = "parent_id"
opensearch_family_tree_key_name = "family_tree"


In [21]:
from local_utils.proc_docs import create_parent_chunk, create_child_chunk

## Parent Chunking

create_parent_chunk() 아래와 같은 작업을 합니다.
- all_docs 에 있는 문서를 parent_chunk_size 만큼으로 청킹 합니다.
- Parent Chunk 에 두개의 메타 데이타를 생성 합니다.
    - family_tree: parent
    - parent_id : None

In [22]:
parent_chunk_docs = create_parent_chunk(all_docs, opensearch_parent_key_name, 
                                        opensearch_family_tree_key_name,parent_chunk_size, parent_chunk_overlap)
print(f"Number of parent_chunk_docs= {len(parent_chunk_docs)}")


Number of parent_chunk_docs= 2325


In [23]:
parent_chunk_docs[0:1]

[Document(page_content='How-to videos. Contains videos on how to use Knox E-FOTA. This section contains videos on how to use Knox E-FOTA. Getting started with Knox E-FOTA This video walks you through the Knox E-FOTA console and demonstrates how you can register a reseller, approve a device, create a campaign, assign a campaign, and monitor device status. Creating a campaign on Knox E-FOTA The following video provides in-depth information on how to create and apply a Knox E-FOTA campaign to your Samsung devices. Connecting Knox E-FOTA to VMware Workspace ONE The following video describes the simple steps of connecting Knox E-FOTA with VMware Workspace ONE, while adding device groups from Workspace ONE.', metadata={'source': 'all_processed_data.json', 'seq_num': 1, 'title': 'How-to videos', 'url': 'https://docs.samsungknox.com/admin/efota-one/how-to-videos', 'project': 'EFOTA', 'last_updated': '2023-09-27', 'family_tree': 'parent', 'parent_id': None})]

OpenSearch 에 parent chunk 삽입

In [24]:
%%time

parent_ids = vector_db.add_documents(
                        documents = parent_chunk_docs, 
                        vector_field = "vector_field",
                        bulk_size = 1000000
                    )



CPU times: user 5.1 s, sys: 297 ms, total: 5.4 s
Wall time: 4min 49s


In [25]:
total_count_docs = opensearch_utils.get_count(os_client, index_name)
print("total count docs: ", total_count_docs)


total count docs:  {'count': 2325, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}}


삽입된 Parent Chunk 의 첫번째를 확인 합니다. family_tree, parent_id 의 값을 확인 하세요.

In [26]:
def show_opensearch_doc_info(response):
    print("opensearch document id:" , response["_id"])
    print("family_tree:" , response["_source"]["metadata"]["family_tree"])
    print("parent document id:" , response["_source"]["metadata"]["parent_id"])
    print("parent document text: \n" , response["_source"]["text"])

response = opensearch_utils.get_document(os_client, doc_id = parent_ids[0], index_name = index_name)
show_opensearch_doc_info(response)    

opensearch document id: dd95cc56-cecc-42e3-81df-abbefee84d80
family_tree: parent
parent document id: None
parent document text: 
 How-to videos. Contains videos on how to use Knox E-FOTA. This section contains videos on how to use Knox E-FOTA. Getting started with Knox E-FOTA This video walks you through the Knox E-FOTA console and demonstrates how you can register a reseller, approve a device, create a campaign, assign a campaign, and monitor device status. Creating a campaign on Knox E-FOTA The following video provides in-depth information on how to create and apply a Knox E-FOTA campaign to your Samsung devices. Connecting Knox E-FOTA to VMware Workspace ONE The following video describes the simple steps of connecting Knox E-FOTA with VMware Workspace ONE, while adding device groups from Workspace ONE.


## Child Chunking

### Child Chunk 생성

아래의 create_child_chunk() 는 다음과 같은 작업을 합니다.
- parent_chunk_docs 각각에 대해서 Child Chunk 를 생성 합니다. 
- Child Chunk 에 두개의 메타 데이타를 생성 합니다.
    - family_tree: child
    - parent_id : parent 에 대한 OpenSearch document id

In [27]:
# child_chunk_docs = create_child_chunk(parent_chunk_docs[0:1], parent_ids)
child_chunk_docs = create_child_chunk(child_chunk_size, child_chunk_overlap, parent_chunk_docs, parent_ids, 
                                      opensearch_parent_key_name, opensearch_family_tree_key_name)
print(f"Number of child_chunk_docs= {len(child_chunk_docs)}")


Number of child_chunk_docs= 6884


### 생성된 Child 와 이에 대한 Parent 정보 확인

Child Chunk 한개에 대한 정보를 확인 합니다.

In [28]:
child_chunk_docs[0:1]

[Document(page_content='How-to videos. Contains videos on how to use Knox E-FOTA. This section contains videos on how to use Knox E-FOTA. Getting started with Knox E-FOTA This video walks you through the Knox E-FOTA console and demonstrates how you can register a reseller, approve a device, create a campaign, assign a campaign, and monitor device status. Creating a campaign on Knox E-FOTA The following video provides in-depth information on how to create and apply a Knox E-FOTA campaign to your Samsung devices. Connecting Knox E-FOTA to VMware Workspace ONE The following video describes the simple steps of connecting Knox E-FOTA with VMware Workspace ONE, while adding device groups from Workspace ONE.', metadata={'source': 'all_processed_data.json', 'seq_num': 1, 'title': 'How-to videos', 'url': 'https://docs.samsungknox.com/admin/efota-one/how-to-videos', 'project': 'EFOTA', 'last_updated': '2023-09-27', 'family_tree': 'child', 'parent_id': 'dd95cc56-cecc-42e3-81df-abbefee84d80'})]

Child 에 대한 Parent 정보 확인

In [29]:
parent_id = child_chunk_docs[0].metadata["parent_id"]
print("child's parent_id: ", parent_id)
print("\n###### Search parent in OpenSearch")
response = opensearch_utils.get_document(os_client, doc_id = parent_id, index_name = index_name)
show_opensearch_doc_info(response)    


child's parent_id:  dd95cc56-cecc-42e3-81df-abbefee84d80

###### Search parent in OpenSearch
opensearch document id: dd95cc56-cecc-42e3-81df-abbefee84d80
family_tree: parent
parent document id: None
parent document text: 
 How-to videos. Contains videos on how to use Knox E-FOTA. This section contains videos on how to use Knox E-FOTA. Getting started with Knox E-FOTA This video walks you through the Knox E-FOTA console and demonstrates how you can register a reseller, approve a device, create a campaign, assign a campaign, and monitor device status. Creating a campaign on Knox E-FOTA The following video provides in-depth information on how to create and apply a Knox E-FOTA campaign to your Samsung devices. Connecting Knox E-FOTA to VMware Workspace ONE The following video describes the simple steps of connecting Knox E-FOTA with VMware Workspace ONE, while adding device groups from Workspace ONE.


### OpenSearch 에 Child 삽입

In [30]:
%%time

child_ids = vector_db.add_documents(
                        documents = child_chunk_docs, 
                        vector_field = "vector_field",
                        bulk_size = 1000000
                    )

print("length of child_ids: ", len(child_ids))

length of child_ids:  6884
CPU times: user 14.9 s, sys: 764 ms, total: 15.7 s
Wall time: 11min 38s


In [31]:
total_count_docs = opensearch_utils.get_count(os_client, index_name)
print("total count docs: ", total_count_docs)


total count docs:  {'count': 9209, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}}


In [32]:
response = opensearch_utils.get_document(os_client, doc_id = child_ids[0], index_name = index_name)
show_opensearch_doc_info(response)    

opensearch document id: a08fd45b-acca-4b24-8840-a8c6169c5879
family_tree: child
parent document id: dd95cc56-cecc-42e3-81df-abbefee84d80
parent document text: 
 How-to videos. Contains videos on how to use Knox E-FOTA. This section contains videos on how to use Knox E-FOTA. Getting started with Knox E-FOTA This video walks you through the Knox E-FOTA console and demonstrates how you can register a reseller, approve a device, create a campaign, assign a campaign, and monitor device status. Creating a campaign on Knox E-FOTA The following video provides in-depth information on how to create and apply a Knox E-FOTA campaign to your Samsung devices. Connecting Knox E-FOTA to VMware Workspace ONE The following video describes the simple steps of connecting Knox E-FOTA with VMware Workspace ONE, while adding device groups from Workspace ONE.


# 7. 검색 테스트

## Lexical 검색

In [33]:
q = "'how to add image"
query ={'query': 
        {'bool': {'must': 
                  [{'match': 
                    {'text': 
                     {'query': "{q}", 'minimum_should_match': '0%', 'operator': 'or'}}}], 
                  'filter': {
                    "term": {
                      "metadata.family_tree": "child"
                    }                      
                  }
                 }
        }
       }
pprint(query)

{'query': {'bool': {'filter': {'term': {'metadata.family_tree': 'child'}},
                    'must': [{'match': {'text': {'minimum_should_match': '0%',
                                                 'operator': 'or',
                                                 'query': '{q}'}}}]}}}


In [34]:
# query = "how to add image"
# query = opensearch_utils.get_query(
#     query=query
# )

response = opensearch_utils.search_document(os_client, query, index_name)
opensearch_utils.parse_keyword_response(response, show_size=3)

# of searched docs:  10
# of display: 3
---------------------
_id in index:  c52ec808-e4c4-4d9f-aacd-f3d3af71f24d
9.231803
. For more information about purchasing licenses, contact the Knox Manage support team. Licenses are granted and revoked in the order in which the devices were enrolled. Attempting to register devices in excess of the device count will result in failure. License FAQs , Q - What happens when my Knox Manage trial license expires? A - You can obtain a retail or another trial Knox Suite license, when your trial license expires. Q - How are Knox Manage licenses allocated to devices? A - Licenses with shorter expiration dates will be allocated first following the First Expired/First Out (FEFO) method. Q - Once the total number of licenses is increased through renewal or extension, how are they allocated? A - If there are more licenses than devices after the increase, licenses are allocated to all devices in the order in which they were enrolled
{'source': 'all_processed_

## 시맨틱 검색

In [35]:
vector_db.similarity_search(q, k=2)

[Document(page_content='. You can add up to 10 image files in the PNG, JPG, JPEG, or GIF format (animated files are not supported). Each image file must be less than 5 MB. To upload an image file, click Add and select a file. To delete an image file, click next to the name of the uploaded image file. Note The device control command must be transferred to the device to apply an image file to it. &gt;&gt;&gt; Video Select a video file for the screen saver. You can add only one video file in the MP4 or MKV format. The video file must be less than 50 MB. To upload a video file, click Add and select a file. To delete a video file, click next to the name of the uploaded video file. Note The device control command must be transferred to the device to apply a video to it. &gt; Session timeout Allows the use of the session timeout feature for the Kiosk Browser', metadata={'source': 'all_processed_data.json', 'seq_num': 893, 'title': 'Android Enterprise policies', 'url': 'https://docs.samsungkno

# 8. 검증 인덱스 생성

## Index 이름 결정

In [36]:
eval_index_name = "v25-genai-poc-knox-eval-parent-doc-retriever"

## Sampling

In [37]:
import random
def get_sampling_doc(seed, ratio, docs):

    random.seed(seed)
    
    eval_docs = docs[:int(len(docs)*ratio)]
    
    return eval_docs
    
eval_docs = get_sampling_doc(seed=200, ratio=0.05, docs= all_docs)
print("eval docs: ", len(eval_docs))
eval_docs[0:2]
    
    

eval docs:  86


[Document(page_content='How-to videos. Contains videos on how to use Knox E-FOTA. This section contains videos on how to use Knox E-FOTA. Getting started with Knox E-FOTA This video walks you through the Knox E-FOTA console and demonstrates how you can register a reseller, approve a device, create a campaign, assign a campaign, and monitor device status. Creating a campaign on Knox E-FOTA The following video provides in-depth information on how to create and apply a Knox E-FOTA campaign to your Samsung devices. Connecting Knox E-FOTA to VMware Workspace ONE The following video describes the simple steps of connecting Knox E-FOTA with VMware Workspace ONE, while adding device groups from Workspace ONE.', metadata={'source': 'all_processed_data.json', 'seq_num': 1, 'title': 'How-to videos', 'url': 'https://docs.samsungknox.com/admin/efota-one/how-to-videos', 'project': 'EFOTA', 'last_updated': '2023-09-27'}),
 Document(page_content='Knox E-FOTA. Knox E-FOTA enables enterprise IT admins t

## 오픈 서치 인덱스 유무에 따라 삭제
오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [38]:
index_exists = opensearch_utils.check_if_index_exists(
    os_client,
    eval_index_name
)

if index_exists:
    opensearch_utils.delete_index(
        os_client,
        eval_index_name
    )
    
opensearch_utils.create_index(os_client, eval_index_name, index_body)
index_info = os_client.indices.get(index=eval_index_name)
print("Index is created")
pprint(index_info)    

index_name=v25-genai-poc-knox-eval-parent-doc-retriever, exists=False

Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'v25-genai-poc-knox-eval-parent-doc-retriever'}
Index is created
{'v25-genai-poc-knox-eval-parent-doc-retriever': {'aliases': {},
                                                  'mappings': {'properties': {'metadata': {'properties': {'family_tree': {'type': 'keyword'},
                                                                                                          'last_updated': {'type': 'date'},
                                                                                                          'parent_id': {'type': 'keyword'},
                                                                                                          'project': {'type': 'keyword'},
                                                                                                          'seq_num': {'type': 'long'},
                           

## 검증 인덱스 연결 오브젝트 생성

In [39]:
eval_vector_db = OpenSearchVectorSearch(
    index_name= eval_index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2",
    bulk_size=100000,
    timeout=60    
)
eval_vector_db

## Parent Chunking

In [40]:
parent_chunk_docs = create_parent_chunk(eval_docs, opensearch_parent_key_name, 
                                        opensearch_family_tree_key_name,parent_chunk_size, 
                                        parent_chunk_overlap)
print(f"Number of parent_chunk_docs= {len(parent_chunk_docs)}")


Number of parent_chunk_docs= 107


In [41]:
parent_chunk_docs[0:1]

[Document(page_content='How-to videos. Contains videos on how to use Knox E-FOTA. This section contains videos on how to use Knox E-FOTA. Getting started with Knox E-FOTA This video walks you through the Knox E-FOTA console and demonstrates how you can register a reseller, approve a device, create a campaign, assign a campaign, and monitor device status. Creating a campaign on Knox E-FOTA The following video provides in-depth information on how to create and apply a Knox E-FOTA campaign to your Samsung devices. Connecting Knox E-FOTA to VMware Workspace ONE The following video describes the simple steps of connecting Knox E-FOTA with VMware Workspace ONE, while adding device groups from Workspace ONE.', metadata={'source': 'all_processed_data.json', 'seq_num': 1, 'title': 'How-to videos', 'url': 'https://docs.samsungknox.com/admin/efota-one/how-to-videos', 'project': 'EFOTA', 'last_updated': '2023-09-27', 'family_tree': 'parent', 'parent_id': None})]

In [42]:
%%time

parent_ids = eval_vector_db.add_documents(
                        documents = parent_chunk_docs, 
                        vector_field = "vector_field",
                        bulk_size = 1000000
                    )



CPU times: user 260 ms, sys: 7.27 ms, total: 267 ms
Wall time: 13.4 s


In [43]:
total_count_docs = opensearch_utils.get_count(os_client, eval_index_name)
print("total count docs: ", total_count_docs)


total count docs:  {'count': 107, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}}


In [44]:
response = opensearch_utils.get_document(os_client, doc_id = parent_ids[0], index_name = eval_index_name)
show_opensearch_doc_info(response)    

opensearch document id: 8b4691d4-ebd7-4198-9055-d19f66a7851f
family_tree: parent
parent document id: None
parent document text: 
 How-to videos. Contains videos on how to use Knox E-FOTA. This section contains videos on how to use Knox E-FOTA. Getting started with Knox E-FOTA This video walks you through the Knox E-FOTA console and demonstrates how you can register a reseller, approve a device, create a campaign, assign a campaign, and monitor device status. Creating a campaign on Knox E-FOTA The following video provides in-depth information on how to create and apply a Knox E-FOTA campaign to your Samsung devices. Connecting Knox E-FOTA to VMware Workspace ONE The following video describes the simple steps of connecting Knox E-FOTA with VMware Workspace ONE, while adding device groups from Workspace ONE.


## Child Chunking

In [45]:
# child_chunk_docs = create_child_chunk(parent_chunk_docs[0:1], parent_ids)
child_chunk_docs = create_child_chunk(child_chunk_size, child_chunk_overlap, parent_chunk_docs, 
                                      parent_ids, 
                                      opensearch_parent_key_name, opensearch_family_tree_key_name)
print(f"Number of child_chunk_docs= {len(child_chunk_docs)}")


Number of child_chunk_docs= 308


In [46]:
child_chunk_docs[0:1]

[Document(page_content='How-to videos. Contains videos on how to use Knox E-FOTA. This section contains videos on how to use Knox E-FOTA. Getting started with Knox E-FOTA This video walks you through the Knox E-FOTA console and demonstrates how you can register a reseller, approve a device, create a campaign, assign a campaign, and monitor device status. Creating a campaign on Knox E-FOTA The following video provides in-depth information on how to create and apply a Knox E-FOTA campaign to your Samsung devices. Connecting Knox E-FOTA to VMware Workspace ONE The following video describes the simple steps of connecting Knox E-FOTA with VMware Workspace ONE, while adding device groups from Workspace ONE.', metadata={'source': 'all_processed_data.json', 'seq_num': 1, 'title': 'How-to videos', 'url': 'https://docs.samsungknox.com/admin/efota-one/how-to-videos', 'project': 'EFOTA', 'last_updated': '2023-09-27', 'family_tree': 'child', 'parent_id': '8b4691d4-ebd7-4198-9055-d19f66a7851f'})]

In [47]:
parent_id = child_chunk_docs[0].metadata["parent_id"]
print("child's parent_id: ", parent_id)
print("\n###### Search parent in OpenSearch")
response = opensearch_utils.get_document(os_client, doc_id = parent_id, index_name = eval_index_name)
show_opensearch_doc_info(response)    


child's parent_id:  8b4691d4-ebd7-4198-9055-d19f66a7851f

###### Search parent in OpenSearch
opensearch document id: 8b4691d4-ebd7-4198-9055-d19f66a7851f
family_tree: parent
parent document id: None
parent document text: 
 How-to videos. Contains videos on how to use Knox E-FOTA. This section contains videos on how to use Knox E-FOTA. Getting started with Knox E-FOTA This video walks you through the Knox E-FOTA console and demonstrates how you can register a reseller, approve a device, create a campaign, assign a campaign, and monitor device status. Creating a campaign on Knox E-FOTA The following video provides in-depth information on how to create and apply a Knox E-FOTA campaign to your Samsung devices. Connecting Knox E-FOTA to VMware Workspace ONE The following video describes the simple steps of connecting Knox E-FOTA with VMware Workspace ONE, while adding device groups from Workspace ONE.


In [48]:
%%time

child_ids = eval_vector_db.add_documents(
                        documents = child_chunk_docs, 
                        vector_field = "vector_field",
                        bulk_size = 1000000
                    )

print("length of child_ids: ", len(child_ids))

length of child_ids:  308
CPU times: user 677 ms, sys: 30.8 ms, total: 708 ms
Wall time: 30.8 s


In [49]:
total_count_docs = opensearch_utils.get_count(os_client, eval_index_name)
print("total count docs: ", total_count_docs)


total count docs:  {'count': 415, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}}


In [50]:
response = opensearch_utils.get_document(os_client, doc_id = child_ids[5], index_name = eval_index_name)
show_opensearch_doc_info(response)    

opensearch document id: 59b063fc-8c7d-4415-b12f-54d480c88541
family_tree: child
parent document id: ba9b5904-4089-4018-800b-70a772e05ded
parent document text: 
 . Typically, the latest firmware updates are pushed to devices by their service provider through a Samsung Business-To-Consumer (B2C) FOTA server. The problem with this is that the latest firmware isn't always compatible with a company's in-house apps. Without Knox E-FOTA, companies can only address this problem by doing the following: Always update to the latest OS version, Block all OS updates using their EMM Knox E-FOTA"allows admins to select a firmware version to deploy, even if it's not the latest version. Devices are then locked to that version. When admins have performed compatibility testing on a later firmware version, they can then update devices to the tested version. With Knox E-FOTA, businesses push firmware updates from a Samsung Business-To-Business (B2B) FOTA"server There are exceptions, such as AT&T and Verizo

In [51]:
parent_id = response["_source"]["metadata"]["parent_id"]
print("child's parent_id: ", parent_id)
print("\n###### Search parent in OpenSearch")
response = opensearch_utils.get_document(os_client, doc_id = parent_id, index_name = eval_index_name)
show_opensearch_doc_info(response)    


child's parent_id:  ba9b5904-4089-4018-800b-70a772e05ded

###### Search parent in OpenSearch
opensearch document id: ba9b5904-4089-4018-800b-70a772e05ded
family_tree: parent
parent document id: None
parent document text: 
 Knox E-FOTA. Knox E-FOTA enables enterprise IT admins to remotely deploy OS versions and security updates to corporate devices without requiring user interaction. Test updates before deployment to verify compatibility between in-house apps and new OS versions, all while increasing the security of enterprise devices by ensuring the latest security patches are deployed on a schedule. Try for free Audience This document is intended for: System Security Architects - Understand how Knox E-FOTA works, and how you can use it to update fleets of enterprise devices. IT Admins - Learn to manage over-the-air updates for enterprise devices. Try the solution Streamline your mobile update workflow using Knox E-FOTA. Enable version control management, and deploy OS updates by setti

## 검색 테스트

In [52]:
q = "'how to use barcode"
query ={'query': 
        {'bool': {'must': 
                  [{'match': 
                    {'text': 
                     {'query': f"{q}", 'minimum_should_match': '0%', 'operator': 'or'}}}], 
                  'filter': {
                    "term": {
                      "metadata.family_tree": "child"
                    }                      
                  }
                 }
        }
       }
pprint(query)

{'query': {'bool': {'filter': {'term': {'metadata.family_tree': 'child'}},
                    'must': [{'match': {'text': {'minimum_should_match': '0%',
                                                 'operator': 'or',
                                                 'query': "'how to use "
                                                          'barcode'}}}]}}}


In [53]:
response = opensearch_utils.search_document(os_client, query, eval_index_name)
opensearch_utils.parse_keyword_response(response, show_size=3)

# of searched docs:  10
# of display: 3
---------------------
_id in index:  92e9ae84-5d9f-470f-a2d3-9edbb4e39d42
4.194215
How-to videos. Contains videos on how to use Knox E-FOTA. This section contains videos on how to use Knox E-FOTA. Getting started with Knox E-FOTA This video walks you through the Knox E-FOTA console and demonstrates how you can register a reseller, approve a device, create a campaign, assign a campaign, and monitor device status. Creating a campaign on Knox E-FOTA The following video provides in-depth information on how to create and apply a Knox E-FOTA campaign to your Samsung devices. Connecting Knox E-FOTA to VMware Workspace ONE The following video describes the simple steps of connecting Knox E-FOTA with VMware Workspace ONE, while adding device groups from Workspace ONE.
{'source': 'all_processed_data.json', 'seq_num': 1, 'title': 'How-to videos', 'url': 'https://docs.samsungknox.com/admin/efota-one/how-to-videos', 'project': 'EFOTA', 'last_updated': '2023-0

In [54]:
eval_vector_db.similarity_search(q, k=2)

[Document(page_content='. Sign in to the Knox Admin Portal. In the left sidebar, click Knox E-FOTA. 2. Go to Devices > All Devices. 3. Click the device ID of the device you want to view. 4. In the Device details popup, click View device log. Tag a device Tagging is useful if you want to label devices and easily search for them in the device list. For example, you can label devices by their business function or by the location of their device users. This section describes how to tag a single device. If you want to tag multiple devices at once with the same tags, see Perform common operations on devices. 1. Sign in to the Knox Admin Portal. In the left sidebar, click Knox E-FOTA. 2. Go to Devices > All Devices. 3. Click the device ID of the device you want to tag. 4. In the Device details dialog, under the Tags field, enter the labels to associate with the device. For example: sales. 5. Click Save.', metadata={'source': 'all_processed_data.json', 'seq_num': 25, 'title': 'Manage devices',